In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('E:/MSFT.csv')

In [3]:
df.head(5)

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019",139.94,139.14,140.40,138.69,37.28M
1,"Oct 23, 2019",137.24,136.88,137.45,135.61,31.38M
2,"Oct 22, 2019",136.37,138.97,140.01,136.26,27.94M
3,"Oct 21, 2019",138.43,138.45,138.50,137.01,20.72M
4,"Oct 18, 2019",137.41,139.76,140.00,136.56,32.28M


In [4]:
df.describe()

,Close Price,Open Price,High Price,Low Price
count,503.000000,503.000000,503.000000,503.000000
mean,109.665805,109.707575,110.626839,108.608628
std,17.381956,17.414436,17.427872,17.310675
min,78.630000,78.580000,79.100000,78.010000
25%,95.385000,95.795000,96.505000,94.125000
50%,107.220000,107.460000,108.300000,105.960000
75%,124.245000,124.105000,125.470000,123.445000
max,141.570000,141.500000,142.370000,140.300000


In [5]:
# 1.2 Calculate the maximum, minimum and mean price for the last 90 days (CLOSE PRICE)
print('Maximum:',df['Close Price'].head(90).max())
print('Minimum:',df['Close Price'].head(90).min())
print('Mean:',df['Close Price'].head(90).mean())

Maximum: 141.57
Minimum: 132.21
Mean: 137.44722222222225


In [7]:
# 1.3 Analyze data types and convert 'Date' column to 'datetime64' datatype
df.dtypes

Date                      object
Close Price              float64
Open Price               float64
High Price               float64
Low Price                float64
Total Traded Quantity     object
dtype: object

In [8]:
df.Date = df.Date.astype('datetime64')
df.dtypes

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity            object
dtype: object

In [9]:
# Subtracting the minimum value from the maximum value of 'Date'
df.Date.max() - df.Date.min()

Timedelta('729 days 00:00:00')

In [12]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [10]:
df['TTQ'] = df['Total Traded Quantity'].str.replace('M','').astype('float').apply(lambda x: x * 10**6).astype('float')

In [13]:
# 1.4 Create a seperate array an calculate VWAP for each month
gb = df.groupby(['Month','Year'])
vwap = []
for m,val in gb:
    temp = 0
    for ind,row in val.iterrows():
        avg = (row['High Price'] + row['Low Price'] + row['Close Price'] ) /3

        temp += avg * row['TTQ']
    vwap.append(temp/val['TTQ'].sum())
    

print(vwap)

[90.49082362501815, 103.86149290581538, 90.90014515321474, 107.64925325090599, 92.7009005421101, 115.30095099853432, 93.03613696675843, 123.2521827961954, 96.7352229420233, 126.27188096195314, 100.48313855974864, 131.76528925348862, 105.27748434299093, 137.97440076220352, 108.54960783454884, 136.07264453589704, 112.2831199028284, 138.20432452614244, 82.61399049968027, 108.1771080037949, 138.06973560543847, 83.68054749715586, 107.06279390733256, 84.64624522716547, 103.84759062556262]


In [16]:
# 1.5 Average price and Profit loss
def calc_avg_price(n):
    return df['Close Price'].head(n).mean()

def profit_loss(n):
    final = df.loc[0]['Close Price']
    initial = df.loc[n-1]['Close Price']
    if final > initial:
        ans = "Profit"
        value = (final-initial)/initial * 100
    else:
        ans = "Loss"
        value = ((initial - final)/initial) * 100
    return ans, value

input_values = [7,14,30,90,180,365]
for val in input_values:
    print('For last',val,'days')
    print('Average Price:',calc_avg_price(val))
    ans, percent = profit_loss(val)
    print(ans,':',percent,'%')
    print('-------')

For last 7 days
Average Price: 138.4985714285714
Loss : 0.33473399330531933 %
-------
For last 14 days
Average Price: 138.60142857142858
Profit : 2.056592765460905 %
-------
For last 30 days
Average Price: 138.293
Profit : 1.9079522283716899 %
-------
For last 90 days
Average Price: 137.44722222222225
Profit : 3.132139435477928 %
-------
For last 180 days
Average Price: 129.03877777777777
Profit : 32.43115359136935 %
-------
For last 365 days
Average Price: 117.27145205479452
Profit : 43.79367036580354 %
-------


In [17]:
# 1.6 add column percent change using pct_change()
df['Day_Perc_Change'] = df['Close Price'].pct_change()
df['Day_Perc_Change'].fillna(0,inplace = True)
df.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,TTQ,Month,Year,Day_Perc_Change
0,2019-10-24,139.94,139.14,140.40,138.69,37.28M,37280000.0,10,2019,0.000000
1,2019-10-23,137.24,136.88,137.45,135.61,31.38M,31380000.0,10,2019,-0.019294
2,2019-10-22,136.37,138.97,140.01,136.26,27.94M,27940000.0,10,2019,-0.006339
3,2019-10-21,138.43,138.45,138.50,137.01,20.72M,20720000.0,10,2019,0.015106
4,2019-10-18,137.41,139.76,140.00,136.56,32.28M,32280000.0,10,2019,-0.007368


In [18]:
# 1.7 Add Trend column according to the given values
def trend(x):
    if x<=0.5 and x>-0.5:
        return 'Slight or No change'
    elif x>0.5 and x<=1:
        return 'Slight positive'
    elif x>-0.5 and x<=-1:
        return 'Slight negative'
    elif x>1 and x<=3:
        return 'Positive'
    elif x>-1 and x<=-3:
        return 'Negative'
    elif x>3 and x<=7:
        return 'Among top gainers'
    elif x>-3 and x<=-7:
        return 'Among top losers'
    elif x>7:
        return 'Bull run'
    elif x<-7:
        return 'Bear Drop'

df['Trend']=df['Day_Perc_Change'].map(lambda a: trend(a))
df.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,TTQ,Month,Year,Day_Perc_Change,Trend
0,2019-10-24,139.94,139.14,140.40,138.69,37.28M,37280000.0,10,2019,0.000000,Slight or No change
1,2019-10-23,137.24,136.88,137.45,135.61,31.38M,31380000.0,10,2019,-0.019294,Slight or No change
2,2019-10-22,136.37,138.97,140.01,136.26,27.94M,27940000.0,10,2019,-0.006339,Slight or No change
3,2019-10-21,138.43,138.45,138.50,137.01,20.72M,20720000.0,10,2019,0.015106,Slight or No change
4,2019-10-18,137.41,139.76,140.00,136.56,32.28M,32280000.0,10,2019,-0.007368,Slight or No change


In [20]:
# 1.8 Calc average and median for groups by Trend
gb_trend = df.groupby('Trend')
df['TTQ'].astype('int')
for grp,val in gb_trend:
    print(grp)
    print('Mean:',val['TTQ'].mean())
    print('Median',val['TTQ'].median())

Slight or No change
Mean: 28324174.950298212
Median 25770000.0


In [23]:
# 1.9 Save dataframe to csv
df.to_csv('week2.csv',index = False)